In [1]:
# Load required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pymo.parsers import BVHParser
from pymo.viz_tools import *
from pymo.preprocessing import *
from sklearn.pipeline import Pipeline

from sklearn import preprocessing

import statsmodels.api as sm
import math

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [2]:
from tensorflow import keras
import pickle 
from keras.utils import np_utils
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input, BatchNormalization, \
    concatenate, Activation, dot, Lambda, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import KFold

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13500261999126513256
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6252920832
locality {
  bus_id: 1
  links {
  }
}
incarnation: 22547258003437934
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Super with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [4]:
import os
import random
import numpy as np
import tensorflow as tf

SEED = 40  # Replace with any fixed number

# Set seed for reproducibility
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [5]:
# Load required libraries
import numpy as np
import os
import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

# Paths to the directories containing train and test data
train_data_path = r'D:\PHD\NEW_November2024\EAWS_npy_final\separate_npy_per_iterationERGD\train'
test_data_path = r'D:\PHD\NEW_November2024\EAWS_npy_final\separate_npy_per_iterationERGD\test'

# Function to read .npy files from a given directory
def read_npy_files(directory_path):
    file_data = {}
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.npy'):
            file_path = os.path.join(directory_path, file_name)
            file_data[file_name] = np.load(file_path)
    return file_data

# Read train and test data
train_data = read_npy_files(train_data_path)
#test_data = read_npy_files(test_data_path)
test_data_nopad = read_npy_files(test_data_path)


# Find the maximum number of frames among all .npy files
max_frames = max([data.shape[0] for data in train_data.values()] + [data.shape[0] for data in test_data_nopad.values()])

# Pad all .npy files with zeros to match the maximum number of frames
def pad_npy_files(data_dict, max_frames):
    padded_data = {}
    for file_name, data in data_dict.items():
        num_frames, num_joints = data.shape
        if num_frames < max_frames:
            # Pad with zeros to match max_frames
            padded_array = np.zeros((max_frames, num_joints))
            padded_array[:num_frames, :] = data
            padded_data[file_name] = padded_array
        else:
            padded_data[file_name] = data
    return padded_data

# Pad train and test data
train_data_padded = pad_npy_files(train_data, max_frames)
test_data_padded = pad_npy_files(test_data_nopad, max_frames)

# Extract labels from file names
def extract_labels(file_names):
    labels = []
    for file_name in file_names:
        class_name = file_name.split('_', 2)[-1].rsplit('_', 1)[0]    # Extract class from the file name
        print(class_name)
        labels.append(class_name)
    return labels

# Prepare train and test datasets
train_labels = extract_labels(train_data_padded.keys())
test_labels = extract_labels(test_data_padded.keys())

# Convert data to lists for easy concatenation
train_data_list = list(train_data_padded.values())
test_data_list = list(test_data_padded.values())

# Check train and test data shapes
for i, data in enumerate(train_data_list):
    print(f"Train data {i} shape: {data.shape}")
for i, data in enumerate(test_data_list):
    print(f"Test data {i} shape: {data.shape}")

# Concatenate train and test data
#initial correct all_data = train_data_list + test_data_list
all_data = train_data_list + test_data_list 
#initial correct all_labels = train_labels + test_labels
all_labels = train_labels + test_labels


# Convert to numpy arrays for training
X_data = np.array(all_data)  # Shape: [number of iterations, number of frames, number of joints]

# Encode labels as categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
y_data = label_encoder.fit_transform(all_labels)
y_data = to_categorical(y_data, num_classes=17)  # Convert labels to categorical format with 17 classes

# Split the data back into train and test sets
num_train = len(train_data_list)
X_train, X_test = X_data[:num_train], X_data[num_train:]
y_train, y_test = y_data[:num_train], y_data[num_train:]

# Print shapes of prepared data
print("X_train shape:", X_train.shape)  # Expected shape: [number of iterations, number of frames, number of joints]
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Data is now ready for training an LSTM network
# You can proceed to define your LSTM model and train it using X_train, y_train

# # Standardize the data
scaler = StandardScaler()

 # Reshape to 2D for the scaler (flatten frames and joints)
X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2])).reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = scaler.transform(X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2])).reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])


Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Kneeling_Upright
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Standing_Ipose
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
Upright
ForwardBending
ForwardBending
ForwardBending
Forward

In [7]:
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LeakyReLU, Input, Add
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# # K-Fold Cross-Validation
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# best_val_accuracy = 0
# best_train_index, best_val_index = None, None

# for train_index, val_index in kf.split(X_data):
#     X_train, X_val = X_data[train_index], X_data[val_index]
#     y_train, y_val = y_data[train_index], y_data[val_index]

#     # Define a CNN Model
#     model = Sequential([
#         Conv1D(64, kernel_size=3, kernel_regularizer=l2(0.01), input_shape=(X_train.shape[1], X_train.shape[2])),
#         LeakyReLU(alpha=0.01),
#         BatchNormalization(),
#         MaxPooling1D(pool_size=2),
#         Dropout(0.3),
#         Conv1D(128, kernel_size=3, kernel_regularizer=l2(0.01)),
#         LeakyReLU(alpha=0.01),
#         BatchNormalization(),
#         MaxPooling1D(pool_size=2),
#         Dropout(0.3),
#         Conv1D(32, kernel_size=3, kernel_regularizer=l2(0.01)),
#         LeakyReLU(alpha=0.01),
#         BatchNormalization(),
#         Dropout(0.3),
#         Flatten(),
#         Dense(64, kernel_regularizer=l2(0.01)),
#         LeakyReLU(alpha=0.01),
#         BatchNormalization(),
#         Dropout(0.3),
#         Dense(17, activation='softmax', kernel_regularizer=l2(0.01))
#     ])

#     # Compile the model
#     learning_rate = 0.001
#     optimizer = Adam(learning_rate=learning_rate)
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#     # Train the Model
#     early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#     history = model.fit(
#         X_train, y_train,
#         validation_data=(X_val, y_val),
#         epochs=100,
#         batch_size=64,
#         callbacks=[early_stopping],
#         verbose=0
#     )

#     # Check validation accuracy
#     val_accuracy = max(history.history['val_accuracy'])
#     if val_accuracy > best_val_accuracy:
#         best_val_accuracy = val_accuracy
#         best_train_index, best_val_index = train_index, val_index

# # Use the best train/validation split for final training
# X_train, X_val = X_data[best_train_index], X_data[best_val_index]
# y_train, y_val = y_data[best_train_index], y_data[best_val_index]

# # Split train data into final train and test sets (use a fixed test set)
# X_test = X_data[num_train:]
# y_test = y_data[num_train:]

# print(f"Final shapes: Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")
# print(f"Labels shapes: Train: {y_train.shape}, Validation: {y_val.shape}, Test: {y_test.shape}")

# #leave one out
# from sklearn.model_selection import LeaveOneOut
# from tensorflow.keras.callbacks import EarlyStopping

# # Initialize Leave-One-Out Cross-Validation
# loo = LeaveOneOut()
# results = []

# for train_index, val_index in loo.split(X_train):
#     X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
#     y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

#     # Define your model here
#     model = Sequential([
#         Conv1D(64, kernel_size=3, kernel_regularizer=l2(0.01), input_shape=(X_train.shape[1], X_train.shape[2])),
#         LeakyReLU(alpha=0.01),
#         BatchNormalization(),
#         MaxPooling1D(pool_size=2),
#         Dropout(0.3),
#         Conv1D(128, kernel_size=3, kernel_regularizer=l2(0.01)),
#         LeakyReLU(alpha=0.01),
#         BatchNormalization(),
#         MaxPooling1D(pool_size=2),
#         Dropout(0.3),
#         Conv1D(32, kernel_size=3, kernel_regularizer=l2(0.01)),
#         LeakyReLU(alpha=0.01),
#         BatchNormalization(),
#         Dropout(0.3),
#         Flatten(),
#         Dense(64, kernel_regularizer=l2(0.01)),
#         LeakyReLU(alpha=0.01),
#         BatchNormalization(),
#         Dropout(0.3),
#         Dense(17, activation='softmax', kernel_regularizer=l2(0.01))
#     ])

#     # Compile the model
#     optimizer = Adam(learning_rate=0.001)
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#     # Train the model
#     early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#     history = model.fit(
#         X_train_fold, y_train_fold,
#         validation_data=(X_val_fold, y_val_fold),
#         epochs=100,
#         batch_size=64,
#         callbacks=[early_stopping],
#         verbose=0
#     )

#     # Evaluate on the validation sample
#     val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
#     results.append(val_accuracy)
#     print(f"Validation Accuracy for current fold: {val_accuracy * 100:.2f}%")

# # Average validation accuracy across all LOOCV iterations
# avg_val_accuracy = np.mean(results)
# print(f"Average Validation Accuracy from LOOCV: {avg_val_accuracy * 100:.2f}%")

# # Train the final model on the entire training set (excluding test data)
# print("\nTraining the final model on the entire training set...")
# model = Sequential([
#     Conv1D(64, kernel_size=3, kernel_regularizer=l2(0.01), input_shape=(X_train.shape[1], X_train.shape[2])),
#     LeakyReLU(alpha=0.01),
#     BatchNormalization(),
#     MaxPooling1D(pool_size=2),
#     Dropout(0.3),
#     Conv1D(128, kernel_size=3, kernel_regularizer=l2(0.01)),
#     LeakyReLU(alpha=0.01),
#     BatchNormalization(),
#     MaxPooling1D(pool_size=2),
#     Dropout(0.3),
#     Conv1D(32, kernel_size=3, kernel_regularizer=l2(0.01)),
#     LeakyReLU(alpha=0.01),
#     BatchNormalization(),
#     Dropout(0.3),
#     Flatten(),
#     Dense(64, kernel_regularizer=l2(0.01)),
#     LeakyReLU(alpha=0.01),
#     BatchNormalization(),
#     Dropout(0.3),
#     Dense(classes, activation='softmax', kernel_regularizer=l2(0.01))
# ])

# # Compile and train the final model
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# history = model.fit(
#     X_train, y_train,
#     epochs=100,
#     batch_size=64,
#     callbacks=[early_stopping],
#     verbose=1
# )

# # Evaluate the final model on the test set
# test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
# print(f"Test Data Evaluation: Accuracy = {test_accuracy * 100:.2f}%, Loss = {test_loss:.4f}")



In [8]:
# # from tensorflow.keras.layers import BatchNormalization, LSTM, Dense, Dropout, Masking
# # from tensorflow.keras.models import Sequential
# # from tensorflow.keras.callbacks import EarlyStopping
# # from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.regularizers import l2

# classes = 17
# max_frames = X_train.shape[1]

# # Define LSTM Model with Regularization, Batch Normalization, and Learning Rate
# print("Step 4: Defining the generalized LSTM model with Masking...")
# model = Sequential([
#     Masking(mask_value=0.0, input_shape=(max_frames, X_train.shape[2])),  # Ignore padding
#     LSTM(256, return_sequences=True, activation='tanh', kernel_regularizer=l2(0.01)),
#     BatchNormalization(),
#     Dropout(0.3),
#     LSTM(128, return_sequences=True, activation='tanh', kernel_regularizer=l2(0.01)),
#     BatchNormalization(),
#     Dropout(0.3),
#     LSTM(64, activation='tanh', kernel_regularizer=l2(0.01)),
#     BatchNormalization(),
#     Dropout(0.3),
#     Dense(classes, activation='softmax', kernel_regularizer=l2(0.01))
# ])

# # Define the optimizer with a learning rate
# learning_rate = 0.001
# optimizer = Adam(learning_rate=learning_rate)

# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

# # Train the Model
# print("Step 5: Training the model with early stopping...")
# early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_val, y_val),
#     epochs=500,
#     batch_size=128,
#     callbacks=[early_stopping],
#     verbose=1
# )
# print("Training complete!")

# # Evaluate the Model
# print("Step 6: Evaluating the model on the test data...")
# test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
# print(f"Test Evaluation: Accuracy = {test_accuracy * 100:.2f}%, Loss = {test_loss:.4f}")

# # Additional: Print Early Stopping Results
# if early_stopping.stopped_epoch > 0:
#     print(f"Early stopping triggered after {early_stopping.stopped_epoch} epochs.")
# else:
#     print("Training ran for the full 500 epochs.")


In [9]:
#with Convolutional Neural Networks
#replaced relu with leakyrelu and added an extra layer for the features

import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LeakyReLU, Input, Add, Attention
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from tensorflow.keras.layers import GlobalMaxPooling1D  # You can replace with GlobalMaxPooling1D if needed

classes = 17
max_frames = X_train.shape[1]

from sklearn.model_selection import train_test_split

# # Split the training data into new train and validation sets
# X_train_new, X_val, y_train_new, y_val = train_test_split(
#     X_train, y_train, test_size=0.2, random_state=41, stratify=y_train.argmax(axis=1)
# )

# Define LSTM Model with Regularization, Batch Normalization, and Learning Rate
print("Step 4: Defining the generalized CNN model with Masking...")

#this is the one the performs best compared to all 
model = Sequential([
    #Conv1D(64, kernel_size=3, input_shape=(max_frames, X_train.shape[2])),
    Conv1D(64, kernel_size=3, kernel_regularizer=l2(0.01), input_shape=(max_frames, X_train.shape[2])),
    LeakyReLU(alpha=0.01),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    #Conv1D(128, kernel_size=3),
    Conv1D(128, kernel_size=3, kernel_regularizer=l2(0.01),),
    LeakyReLU(alpha=0.01),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    #Conv1D(32, kernel_size=3),  # Added additional Conv1D layer

    Conv1D(32, kernel_size=3, kernel_regularizer=l2(0.01)),  # Added additional Conv1D layer
    LeakyReLU(alpha=0.01),
    BatchNormalization(),
    Dropout(0.3),

    # Attention layer
    #Lambda(lambda x: Attention()([x, x])),
    
    Flatten(),
    # Add GlobalAveragePooling1D to handle variable-length sequences and reduce impact of padding
    #GlobalMaxPooling1D(),  # Or GlobalMaxPooling1D()
    Dense(64),
    LeakyReLU(alpha=0.01),
    BatchNormalization(),
    Dropout(0.3),
    Dense(classes, activation='softmax')
])

# model = Sequential([
#     Conv1D(64, kernel_size=3, kernel_regularizer=l2(0.01), input_shape=(max_frames, X_train.shape[2])),
#     LeakyReLU(alpha=0.01),
#     BatchNormalization(),
#     MaxPooling1D(pool_size=2),
#     Dropout(0.3),
#     Conv1D(128, kernel_size=3, kernel_regularizer=l2(0.01)),
#     LeakyReLU(alpha=0.01),
#     BatchNormalization(),
#     MaxPooling1D(pool_size=2),
#     Dropout(0.3),
#     Conv1D(32, kernel_size=3, kernel_regularizer=l2(0.01)),  # Added additional Conv1D layer
#     LeakyReLU(alpha=0.01),
#     BatchNormalization(),
#     Dropout(0.3),
#     Conv1D(16, kernel_size=3, kernel_regularizer=l2(0.01)),  # Extra Conv1D layer
#     LeakyReLU(alpha=0.01),
#     BatchNormalization(),
#     Dropout(0.3),
#     Flatten(),
#     Dense(64, kernel_regularizer=l2(0.01)),
#     LeakyReLU(alpha=0.01),
#     BatchNormalization(),
#     Dropout(0.3),
#     Dense(classes, activation='softmax', kernel_regularizer=l2(0.01))
# ])


# #this is to keep orginally
# # Define the optimizer with a learning rate
# learning_rate = 0.001
# optimizer = Adam(learning_rate=learning_rate)

# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

# # Train the Model
# print("Step 5: Training the model with early stopping...")
# early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_val, y_val),
#     epochs=500,
#     batch_size=128,
#     callbacks=[early_stopping],
#     verbose=1
# )
# print("Training complete!")

# # Evaluate the Model
# print("Step 6: Evaluating the model on the test data...")
# test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
# print(f"Test Evaluation: Accuracy = {test_accuracy * 100:.2f}%, Loss = {test_loss:.4f}")

# # Additional: Print Early Stopping Results
# if early_stopping.stopped_epoch > 0:
#     print(f"Early stopping triggered after {early_stopping.stopped_epoch} epochs.")
# else:
#     print("Training ran for the full 500 epochs.")

Step 4: Defining the generalized CNN model with Masking...


In [10]:
# from sklearn.model_selection import StratifiedKFold
# from tensorflow.keras.models import clone_model
# import numpy as np
# import os
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.layers import BatchNormalization, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LeakyReLU, Input, Add
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.regularizers import l2

# # Number of folds
# n_folds = 5
# kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# # Placeholder for cross-validation results
# cv_accuracies = []
# cv_losses = []

# # Perform k-fold cross-validation
# print(f"Performing {n_folds}-fold cross-validation...")
# for fold, (train_index, val_index) in enumerate(kf.split(X_train, y_train.argmax(axis=1))):
#     print(f"Training on fold {fold + 1}/{n_folds}...")
    
#     # Split data into train and validation sets for this fold
#     X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
#     y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
#     # Clone the model to ensure weights are reset
#     fold_model = clone_model(model)
#     fold_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
#     # Train the model
#     early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True, verbose=0)
#     history = fold_model.fit(
#         X_train_fold, y_train_fold,
#         validation_data=(X_val_fold, y_val_fold),
#         epochs=500,
#         batch_size=128,
#         callbacks=[early_stopping],
#         verbose=0
#     )
    
#     # Evaluate on the validation set
#     val_loss, val_accuracy = fold_model.evaluate(X_val_fold, y_val_fold, verbose=0)
#     cv_accuracies.append(val_accuracy)
#     cv_losses.append(val_loss)
#     print(f"Fold {fold + 1}: Validation Accuracy = {val_accuracy * 100:.2f}%, Loss = {val_loss:.4f}")

# # Cross-validation results
# mean_cv_accuracy = np.mean(cv_accuracies)
# mean_cv_loss = np.mean(cv_losses)
# print(f"\nCross-Validation Results: Mean Accuracy = {mean_cv_accuracy * 100:.2f}%, Mean Loss = {mean_cv_loss:.4f}")




In [11]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import clone_model
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LeakyReLU, Input, Add
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Split the training data into new train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=40, stratify=y_train.argmax(axis=1)
)

# Train on the full training set and evaluate on the test set
print("\nTraining on the full training set and evaluating on the test set...")
#final_model = clone_model(model)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True, verbose=0)


final_history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=500,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=1, 
    #shuffle=False  # Disable shuffling
)

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nFinal Test Results: Accuracy = {test_accuracy * 100:.2f}%, Loss = {test_loss:.4f}")


Training on the full training set and evaluating on the test set...
Epoch 1/500
9/9 [==============================] - 4s 82ms/step - loss: 4.6343 - accuracy: 0.2639 - val_loss: 4.5419 - val_accuracy: 0.2370
Epoch 2/500
9/9 [==============================] - 0s 32ms/step - loss: 3.2215 - accuracy: 0.6877 - val_loss: 3.5828 - val_accuracy: 0.5630
Epoch 3/500
9/9 [==============================] - 0s 33ms/step - loss: 2.7827 - accuracy: 0.7937 - val_loss: 3.1089 - val_accuracy: 0.7259
Epoch 4/500
9/9 [==============================] - 0s 34ms/step - loss: 2.4543 - accuracy: 0.9033 - val_loss: 2.9273 - val_accuracy: 0.7407
Epoch 5/500
9/9 [==============================] - 0s 31ms/step - loss: 2.3075 - accuracy: 0.9312 - val_loss: 2.7646 - val_accuracy: 0.7556
Epoch 6/500
9/9 [==============================] - 0s 32ms/step - loss: 2.1358 - accuracy: 0.9647 - val_loss: 2.6739 - val_accuracy: 0.7556
Epoch 7/500
9/9 [==============================] - 0s 35ms/step - loss: 2.0601 - accuracy: 

In [ ]:
# ###### import numpy as np
# import os
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, accuracy_score
# import seaborn as sns

# # Confusion Matrix and Metrics
# print("Step 7: Generating confusion matrix and metrics...")
# y_pred = model.predict(X_test)
# y_pred_classes = np.argmax(y_pred, axis=1)
# y_true = np.argmax(y_test, axis=1)

# # Confusion Matrix
# conf_matrix = confusion_matrix(y_true, y_pred_classes)
# plt.figure(figsize=(10, 8))
# sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
# plt.xlabel('Predicted Labels')
# plt.ylabel('True Labels')
# plt.title('Confusion Matrix')
# plt.show()

# # Classification Report
# print("Classification Report:")
# print(classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_))

# # Precision, Recall, F1-Score, and Accuracy for each class
# precision, recall, fscore, support = precision_recall_fscore_support(y_true, y_pred_classes, average=None)
# accuracy_per_class = []
# for i in range(classes):
#     class_indices = (y_true == i)
#     class_accuracy = (y_pred_classes[class_indices] == y_true[class_indices]).mean()
#     accuracy_per_class.append(class_accuracy)
#     print(f"Class {label_encoder.classes_[i]} - Precision: {precision[i]:.2f}, Recall: {recall[i]:.2f}, F1-Score: {fscore[i]:.2f}, Accuracy: {class_accuracy:.2f}, Support: {support[i]}")

# # Plot Training and Validation Loss and Accuracy
# print("Step 8: Plotting training history...")
# plt.figure(figsize=(12, 4))

# # Plot loss
# plt.subplot(1, 2, 1)
# plt.plot(final_history.history['loss'], label='Training Loss')
# plt.plot(final_history.history['val_loss'], label='Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.title('Training and Validation Loss')

# # Plot accuracy
# plt.subplot(1, 2, 2)
# plt.plot(final_history.history['accuracy'], label='Training Accuracy')
# plt.plot(final_history.history['val_accuracy'], label='Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.title('Training and Validation Accuracy')

# plt.show()



In [ ]:
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
# import seaborn as sns

# # Define the output directory
# output_dir = r"C:\Users\aimove Gavriela\Desktop\PHD_test results\test_7"
# os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# # Confusion Matrix and Metrics
# print("Step 7: Generating confusion matrix and metrics...")
# y_pred = model.predict(X_test)
# y_pred_classes = np.argmax(y_pred, axis=1)
# y_true = np.argmax(y_test, axis=1)

# # Confusion Matrix
# conf_matrix = confusion_matrix(y_true, y_pred_classes)
# plt.figure(figsize=(10, 8))
# sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
# plt.xlabel('Predicted Labels')
# plt.ylabel('True Labels')
# plt.title('Confusion Matrix')

# # Save confusion matrix plot
# conf_matrix_path = os.path.join(output_dir, "confusion_matrix.png")
# plt.savefig(conf_matrix_path)
# plt.close()
# print(f"Confusion matrix saved to {conf_matrix_path}")

# # Classification Report
# print("Classification Report:")
# class_report = classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_)
# print(class_report)

# # Save classification report to a text file
# class_report_path = os.path.join(output_dir, "classification_report.txt")
# with open(class_report_path, "w") as f:
#     f.write(class_report)
# print(f"Classification report saved to {class_report_path}")

# # Precision, Recall, F1-Score, and Accuracy for each class
# precision, recall, fscore, support = precision_recall_fscore_support(y_true, y_pred_classes, average=None)
# accuracy_per_class = []
# metrics_path = os.path.join(output_dir, "per_class_metrics.txt")

# with open(metrics_path, "w") as f:
#     for i in range(classes):
#         class_indices = (y_true == i)
#         class_accuracy = (y_pred_classes[class_indices] == y_true[class_indices]).mean()
#         accuracy_per_class.append(class_accuracy)
#         metrics_line = (
#             f"Class {label_encoder.classes_[i]} - Precision: {precision[i]:.2f}, "
#             f"Recall: {recall[i]:.2f}, F1-Score: {fscore[i]:.2f}, "
#             f"Accuracy: {class_accuracy:.2f}, Support: {support[i]}\n"
#         )
#         f.write(metrics_line)
#         print(metrics_line.strip())
# print(f"Per-class metrics saved to {metrics_path}")

# # Plot Training and Validation Loss and Accuracy
# print("Step 8: Plotting training history...")
# plt.figure(figsize=(12, 4))

# # Plot loss
# plt.subplot(1, 2, 1)
# plt.plot(final_history.history['loss'], label='Training Loss')
# plt.plot(final_history.history['val_loss'], label='Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.title('Training and Validation Loss')

# # Plot accuracy
# plt.subplot(1, 2, 2)
# plt.plot(final_history.history['accuracy'], label='Training Accuracy')
# plt.plot(final_history.history['val_accuracy'], label='Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.title('Training and Validation Accuracy')

# # Save training history plots
# history_plot_path = os.path.join(output_dir, "training_history.png")
# plt.savefig(history_plot_path)
# plt.close()
# print(f"Training history plots saved to {history_plot_path}")